In [71]:
# DON'T RUN THIS BLOCK AGAIN

import configparser
import json
from datetime import datetime
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

# Reading Configs
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)
await client.start()
print("Client Created")

Please enter your phone (or bot token): +85292816680


Connection error 1 during auth_key gen: ConnectionError: Not connected


Please enter the code you received: 57680
Please enter your password: ········
Signed in successfully as ❣️
Client Created


In [72]:
# DON'T RUN THIS BLOCK AGAIN

# Ensure you're authorized
if not client.is_user_authorized():
    client.send_code_request(phone)
    try:
        client.sign_in(phone, input('Enter the code: '))
    except SessionPasswordNeededError:
        client.sign_in(password=input('Password: '))

from telethon.tl.types import PeerChannel

# user_input_channel = input("enter entity(telegram URL or entity id):")
user_input_channel = '1001546061018'

if user_input_channel.isdigit():
    entity = PeerChannel(-int(user_input_channel))
else:
    entity = user_input_channel

await client.connect()
my_channel = await client.get_entity(entity)

/var/folders/ym/67x0srsx2ln9vy4qtlvtdv500000gn/T/ipykernel_50101/2990546721.py:4: RuntimeWarning: coroutine 'UserMethods.is_user_authorized' was never awaited
  if not client.is_user_authorized():


In [135]:
# Getting Channel messages
from telethon.tl.functions.messages import GetHistoryRequest
import datetime
offset_id = 0
limit = 100
all_messages = []
total_messages = 0
total_count_limit = 300
long_call = 0
otm_call = 0
long_put = 0
otm_put = 0
short_call = 0
short_put = 0
iteration = 1
due_a_object = 0
due_b_object = 0
date_now = datetime.datetime(2022, 5, 25)
date = date_now.strftime('%Y-%m-%d')


def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + datetime.timedelta(days_ahead)


while True:
#     print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
    history = await client(GetHistoryRequest(
        peer=my_channel,
        offset_id=offset_id,
        offset_date=date_now + datetime.timedelta(1),
        add_offset=0,
        limit=limit,
        max_id=0,
        min_id=0,
        hash=0
    ))
    if not history.messages:
        break
    messages = history.messages
    for message in messages:
        all_messages.append(message.to_dict())
    offset_id = messages[len(messages) - 1].id
    total_messages = len(all_messages)
    if total_count_limit != 0 and total_messages >= total_count_limit:
        break
# print("Total Messages:", total_messages)


for item in all_messages:
    content = item['message']
    buy_index = content.index('成交日期') + 5
    buy_date = content[buy_index:buy_index + 10]
    buy_object = datetime.datetime.fromisoformat(buy_date)
    if buy_object == date_now:
        due_index = content.index('到期日:') + 5
        due_date = content[due_index:due_index + 10]
        due_object = datetime.datetime.fromisoformat(due_date)
        next_fri = next_weekday(date_now, 4)
        due = next_fri

        strike_index = content.index('行權價') + 5
        strike = float(content[strike_index:strike_index + 5])
        price_index = content.index('正股股價') + 6
        price = float(content[price_index:price_index + 5])

        if due == due_object:
            if 'LONG CALL' in content:
                long_call += 1
                if price < strike:
                    otm_call += 1
            if 'LONG PUT' in content:
                long_put += 1
                if price > strike:
                    otm_put += 1
            if 'SHORT CALL' in content:
                short_call += 1
            if 'SHORT PUT' in content:
                short_put += 1

# Let Call be 1 and Put be -1
score = (otm_call - otm_put)/(otm_call + otm_put)
print(score)
score2 = (long_call - long_put)/(long_call + long_put)
print(score2)
score3 = (long_call - short_call - (long_put - short_put))/(long_call - short_call + long_put - short_put)
print(score3)
                
print('Option alert of', date,':')
print('\n\033[1mDue',due.strftime('%Y-%m-%d'),':\033[0m')
print('Number of LONG CALL:', long_call)
print('Number of LONG PUT:', long_put)
print('Number of SHORT CALL:', short_call)
print('Number of SHORT PUT:', short_put)
print('Net CALL:', long_call - short_call)
print('Net PUT:', long_put - short_put)
print('Number of OTM LONG CALL:', otm_call)
print('Number of OTM LONG PUT:', otm_put)


0.45454545454545453
0.5555555555555556
2.0
Option alert of 2022-05-25 :

Due 2022-05-27 :
Number of LONG CALL: 14
Number of LONG PUT: 4
Number of SHORT CALL: 8
Number of SHORT PUT: 6
Net CALL: 6
Net PUT: -2
Number of OTM LONG CALL: 8
Number of OTM LONG PUT: 3


In [79]:
# Find correlation between option alert and change in the next day
from pandas_datareader.data import DataReader

df = DataReader('spy', data_source='yahoo', start=start, end=datetime.datetime.now())
close = df.filter(['Close'])
close.index = close.index.strftime('%Y-%m-%d')

close.to_csv('data.csv', sep='\t')

In [81]:
if date_now.weekday() != 4:
    next_day = date_now + datetime.timedelta(1)
else:
    next_day = date_now + datetime.timedelta(3)
next_day = next_day.strftime('%Y-%m-%d')
change = close.loc[next_day] - close.loc[date]


Close    7.940002
dtype: float64

In [76]:
import pandas as pd


if date_now.weekday() != 0:
    end_date = date_now - datetime.timedelta(1)
else:
    end_date = date_now - datetime.timedelta(3)
end_date = date_now.strftime('%Y-%m-%d')

In [77]:
index = close.index[0]

In [78]:
from telethon.tl.functions.messages import GetHistoryRequest
import datetime

change_list, all_messages = [], []
offset_id = 0
limit = 100
total_count_limit = 300000
start = '2018-01-01'
date_start = datetime.datetime(2018, 1, 1)


for i in range(-50,len(close) - 1):
    long_call = 0
    otm_call = 0
    long_put = 0
    otm_put = 0
    short_call = 0
    short_put = 0
    date = close.index[i]
    date_now = datetime.datetime.fromisoformat(date)
    
    change = close.iat[i + 1,0] - close.iloc[i,0]
    change_list.append(change)
    print(change_list)
    
    history = await client(GetHistoryRequest(
        peer=my_channel,
        offset_id=offset_id,
        offset_date=date_now + datetime.timedelta(1),
        add_offset=0,
        limit=limit,
        max_id=0,
        min_id=0,
        hash=0
    ))
    print('history:',history)
    if not history.messages:
        pass
    messages = history.messages
    for message in messages:
        all_messages.append(message.to_dict())
    offset_id = messages[len(messages) - 1].id
    total_messages = len(all_messages)
    if total_count_limit != 0 and total_messages >= total_count_limit:
        pass

        
    for item in all_messages:
        print(item)
        content = item['message']
        print(content)
        while True:
            buy_index = content.index('成交日期') + 5
            buy_date = content[buy_index:buy_index + 10]
            buy_object = datetime.datetime.fromisoformat(buy_date)
            if buy_object == date_now:
                due_index = content.index('到期日:') + 5
                due_date = content[due_index:due_index + 10]
                due_object = datetime.datetime.fromisoformat(due_date)
                next_fri = next_weekday(date_now, 4)
                due = next_fri

                strike_index = content.index('行權價') + 5
                strike = float(content[strike_index:strike_index + 5])
                price_index = content.index('正股股價') + 6
                price = float(content[price_index:price_index + 5])

                if due == due_object:
                    if 'LONG CALL' in content:
                        long_call += 1
                        if price < strike:
                            otm_call += 1
                    if 'LONG PUT' in content:
                        long_put += 1
                        if price > strike:
                            otm_put += 1
                    if 'SHORT CALL' in content:
                        short_call += 1
                    if 'SHORT PUT' in content:
                        short_put += 1

    # Let Call be 1 and Put be -1
    score = (otm_call - otm_put)/(otm_call + otm_put)
    print(score)

[-13.16998291015625]
history: ChannelMessages(pts=11306, count=10226, messages=[MessageService(id=383, peer_id=PeerChannel(channel_id=1546061018), date=datetime.datetime(2022, 5, 4, 9, 4, 5, tzinfo=datetime.timezone.utc), action=MessageActionChatEditTitle(title='💡Topaz 期權異動Alert Bot💡(Testing)'), out=False, mentioned=False, media_unread=False, silent=False, post=True, legacy=False, from_id=None, reply_to=None, ttl_period=None), Message(id=382, peer_id=PeerChannel(channel_id=1546061018), date=datetime.datetime(2022, 5, 4, 9, 3, 41, tzinfo=datetime.timezone.utc), message='::/', out=False, mentioned=False, media_unread=False, silent=False, post=True, from_scheduled=False, legacy=False, edit_hide=False, pinned=False, from_id=None, fwd_from=None, via_bot_id=None, reply_to=None, media=None, reply_markup=None, entities=[], views=148, forwards=0, replies=None, edit_date=None, post_author=None, grouped_id=None, restriction_reason=[], ttl_period=None), Message(id=381, peer_id=PeerChannel(channel_

KeyError: 'message'

In [ ]:
print(content)

Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: attempt to write a readonly database
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: attempt to write a readonly database
Error executing high-level request after reconnect: <class 'sqlite3.OperationalError'>: attempt to write a readonly database
Task exception was never retrieved
future: <Task finished name='Task-949' coro=<UpdateMethods._dispatch_update() done, defined at /Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/updates.py:399> exception=OperationalError('attempt to write a readonly database')>
Traceback (most recent call last):
  File "/Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/updates.py", line 430, in _dispatch_update
    await self.get_me(input_peer=True)
  File "/Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/users.py", line 161, in get_me
    me = (await self(
  File

Task exception was never retrieved
future: <Task finished name='Task-969' coro=<UpdateMethods._dispatch_update() done, defined at /Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/updates.py:399> exception=OperationalError('attempt to write a readonly database')>
Traceback (most recent call last):
  File "/Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/updates.py", line 408, in _dispatch_update
    await self._get_difference(update, channel_id, pts_date)
  File "/Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/updates.py", line 558, in _get_difference
    result = await self(functions.updates.GetChannelDifferenceRequest(
  File "/Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/users.py", line 30, in __call__
    return await self._call(self._sender, request, ordered=ordered)
  File "/Users/frankie/opt/anaconda3/lib/python3.9/site-packages/telethon/client/users.py", line 85, in _call
    self